In [20]:
import pandas as pd 
import numpy as np
from sklearn.metrics import accuracy_score ,roc_auc_score
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression 
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.ensemble import RandomForestClassifier

from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import VotingClassifier

In [21]:
liver=pd.read_csv('Downloads\indian_liver_patient.csv')

In [22]:
liver.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 583 entries, 0 to 582
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Age                         583 non-null    int64  
 1   Gender                      583 non-null    object 
 2   Total_Bilirubin             583 non-null    float64
 3   Direct_Bilirubin            583 non-null    float64
 4   Alkaline_Phosphotase        583 non-null    int64  
 5   Alamine_Aminotransferase    583 non-null    int64  
 6   Aspartate_Aminotransferase  583 non-null    int64  
 7   Total_Protiens              583 non-null    float64
 8   Albumin                     583 non-null    float64
 9   Albumin_and_Globulin_Ratio  579 non-null    float64
 10  Dataset                     583 non-null    int64  
dtypes: float64(5), int64(5), object(1)
memory usage: 50.2+ KB


In [23]:
liver.head()

,Age,Gender,Total_Bilirubin,Direct_Bilirubin,Alkaline_Phosphotase,Alamine_Aminotransferase,Aspartate_Aminotransferase,Total_Protiens,Albumin,Albumin_and_Globulin_Ratio,Dataset
0,65,Female,0.7,0.1,187,16,18,6.8,3.3,0.90,1
1,62,Male,10.9,5.5,699,64,100,7.5,3.2,0.74,1
2,62,Male,7.3,4.1,490,60,68,7.0,3.3,0.89,1
3,58,Male,1.0,0.4,182,14,20,6.8,3.4,1.00,1
4,72,Male,3.9,2.0,195,27,59,7.3,2.4,0.40,1


In [24]:
liver.isnull().sum()

Age                           0
Gender                        0
Total_Bilirubin               0
Direct_Bilirubin              0
Alkaline_Phosphotase          0
Alamine_Aminotransferase      0
Aspartate_Aminotransferase    0
Total_Protiens                0
Albumin                       0
Albumin_and_Globulin_Ratio    4
Dataset                       0
dtype: int64

In [25]:
liver[liver['Albumin_and_Globulin_Ratio'].isnull()]

,Age,Gender,Total_Bilirubin,Direct_Bilirubin,Alkaline_Phosphotase,Alamine_Aminotransferase,Aspartate_Aminotransferase,Total_Protiens,Albumin,Albumin_and_Globulin_Ratio,Dataset
209,45,Female,0.9,0.3,189,23,33,6.6,3.9,NaN,1
241,51,Male,0.8,0.2,230,24,46,6.5,3.1,NaN,1
253,35,Female,0.6,0.2,180,12,15,5.2,2.7,NaN,2
312,27,Male,1.3,0.6,106,25,54,8.5,4.8,NaN,2


In [26]:
from sklearn.impute import SimpleImputer
imp=SimpleImputer(missing_values=np.nan ,strategy='mean')

In [27]:
liver['Albumin_and_Globulin_Ratio']=imp.fit_transform(liver[['Albumin_and_Globulin_Ratio']])

liver['Albumin_and_Globulin_Ratio'].isnull().sum()

0

In [28]:
liver['Gender'].unique()

array(['Female', 'Male'], dtype=object)

In [29]:
liver['Gender']=liver['Gender'].apply(lambda x:1 if x=='Male' else 0)
liver['Gender'].unique()

array([0, 1], dtype=int64)

In [31]:
liver.corr().style.background_gradient(cmap="PuBu")

,Age,Gender,Total_Bilirubin,Direct_Bilirubin,Alkaline_Phosphotase,Alamine_Aminotransferase,Aspartate_Aminotransferase,Total_Protiens,Albumin,Albumin_and_Globulin_Ratio,Dataset
Age,1.000000,0.056560,0.011763,0.007529,0.080425,-0.086883,-0.019910,-0.187461,-0.265924,-0.216089,-0.137351
Gender,0.056560,1.000000,0.089291,0.100436,-0.027496,0.082332,0.080336,-0.089121,-0.093799,-0.003404,-0.082416
Total_Bilirubin,0.011763,0.089291,1.000000,0.874618,0.206669,0.214065,0.237831,-0.008099,-0.222250,-0.206159,-0.220208
Direct_Bilirubin,0.007529,0.100436,0.874618,1.000000,0.234939,0.233894,0.257544,-0.000139,-0.228531,-0.200004,-0.246046
Alkaline_Phosphotase,0.080425,-0.027496,0.206669,0.234939,1.000000,0.125680,0.167196,-0.028514,-0.165453,-0.233960,-0.184866
Alamine_Aminotransferase,-0.086883,0.082332,0.214065,0.233894,0.125680,1.000000,0.791966,-0.042518,-0.029742,-0.002374,-0.163416
Aspartate_Aminotransferase,-0.019910,0.080336,0.237831,0.257544,0.167196,0.791966,1.000000,-0.025645,-0.085290,-0.070024,-0.151934
Total_Protiens,-0.187461,-0.089121,-0.008099,-0.000139,-0.028514,-0.042518,-0.025645,1.000000,0.784053,0.233904,0.035008
Albumin,-0.265924,-0.093799,-0.222250,-0.228531,-0.165453,-0.029742,-0.085290,0.784053,1.000000,0.686322,0.161388
Albumin_and_Globulin_Ratio,-0.216089,-0.003404,-0.206159,-0.200004,-0.233960,-0.002374,-0.070024,0.233904,0.686322,1.000000,0.162319


In [34]:
x=liver.drop('Dataset',axis=1)
y=liver['Dataset'].values

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

In [35]:
X_train.head()

,Age,Gender,Total_Bilirubin,Direct_Bilirubin,Alkaline_Phosphotase,Alamine_Aminotransferase,Aspartate_Aminotransferase,Total_Protiens,Albumin,Albumin_and_Globulin_Ratio
552,45,1,0.6,0.1,196,29,30,5.8,2.9,1.0
280,58,1,0.8,0.2,123,56,48,6.0,3.0,1.0
234,40,0,0.9,0.2,285,32,27,7.7,3.5,0.8
255,38,1,1.7,0.7,859,89,48,6.0,3.0,1.0
438,65,1,0.8,0.2,162,30,90,3.8,1.4,0.5


In [36]:
scaler=StandardScaler()

X_train=scaler.fit_transform(X_train)

X_test=scaler.transform(X_test)

In [37]:
logreg=LogisticRegression(random_state=1)

Knn=KNN(n_neighbors=27)

dt=DecisionTreeClassifier(min_samples_leaf=0.13,random_state=1)

classifiers=[('Logistic Regression',logreg),
            ('K Nearest',Knn),
            ('Classification Tree',dt)]

for clf_name,clf in classifiers:
    clf.fit(X_train,y_train)
    
    y_pred=clf.predict(X_test)
    
    accuracy=accuracy_score(y_pred,y_test)
    
    print('{:20}: {:.3f}'.format(clf_name ,accuracy))

Logistic Regression : 0.736
K Nearest           : 0.736
Classification Tree : 0.715


In [38]:
from sklearn.ensemble import VotingClassifier

vc=VotingClassifier(estimators=classifiers)

vc.fit(X_train,y_train)

y_pred=vc.predict(X_test)

accuracy=accuracy_score(y_pred,y_test)

print('voting: {:.3f}'.format(accuracy))

voting: 0.720
